## **Task 4-1**

In [ ]:
!pip install tensorflow tensorflow-datasets opencv-python pandas

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# -------------------------
# Configuration
# -------------------------
BATCH_SIZE = 32
EPOCHS = 3  # For demonstration; increase for better performance
SEED = 123

# We'll use the "tf_flowers" dataset (5 flower classes)
(ds_train, ds_val), ds_info = tfds.load('tf_flowers', split=['train[:80%]', 'train[80%:]'],
                                          as_supervised=True, with_info=True)
class_names = ds_info.features['label'].names  # e.g., ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
num_classes = len(class_names)
print("Classes:", class_names)

# For each model, we define its input size and its preprocessing function.
models_dict = {
    "VGG16": {
         "model_fn": VGG16,
         "input_size": (224, 224),
         "preprocess": vgg16_preprocess
    },
    "ResNet50": {
         "model_fn": ResNet50,
         "input_size": (224, 224),
         "preprocess": resnet50_preprocess
    },
    "InceptionV3": {
         "model_fn": InceptionV3,
         "input_size": (299, 299),
         "preprocess": inception_preprocess
    }
}

# -------------------------
# Utility Functions
# -------------------------
def preprocess_dataset(ds, input_size, preprocess_fn, batch_size=BATCH_SIZE, shuffle=False):
    """
    Resizes images to input_size, casts them to float32, and applies the given preprocessing function.
    """
    def _preprocess(image, label):
        image = tf.image.resize(image, input_size)
        image = tf.cast(image, tf.float32)
        image = preprocess_fn(image)
        return image, label
    ds = ds.map(_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(1000, seed=SEED)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

def build_model(model_fn, input_size, num_classes):
    """
    Builds a transfer learning model using the specified Keras Applications model.
    (Preprocessing is applied in the dataset pipeline.)
    """
    inp = Input(shape=(*input_size, 3))
    base_model = model_fn(weights='imagenet', include_top=False, input_tensor=inp)
    x = GlobalAveragePooling2D()(base_model.output)
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inp, outputs=out)
    return model

def add_noise_to_image(image, noise_factor=0.2):
    """
    Adds Gaussian noise to an image.
    """
    noisy = image + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=image.shape)
    return np.clip(noisy, 0, 255)

def test_tricky_image(model, input_size, preprocess_fn):
    """
    Takes one sample image from the tf_flowers dataset (as a numpy array),
    resizes it, applies the given preprocessing, and makes a prediction.
    Then it adds noise to the image and predicts again.
    Returns the predicted classes for the original and noisy image.
    """
    # Load one sample (unbatched) from tf_flowers:
    ds_sample = tfds.load('tf_flowers', split='train', as_supervised=True)
    for image, label in ds_sample.take(1):
        sample_image = image.numpy()  # shape (H, W, 3), uint8
        break
    # Resize using OpenCV
    img_resized = cv2.resize(sample_image, input_size)
    img_resized = np.expand_dims(img_resized, axis=0)  # (1, H, W, 3)
    # Preprocess and predict original
    img_preprocessed = preprocess_fn(img_resized.astype(np.float32))
    pred_orig = model.predict(img_preprocessed)
    orig_class = class_names[np.argmax(pred_orig)]
    # Add noise
    noisy = add_noise_to_image(img_resized.astype(np.float32))
    noisy_preprocessed = preprocess_fn(noisy)
    pred_noisy = model.predict(noisy_preprocessed)
    noisy_class = class_names[np.argmax(pred_noisy)]
    return orig_class, noisy_class

# -------------------------
# Main Pipeline
# -------------------------
results_summary = []

for model_name, model_info in models_dict.items():
    print(f"\nProcessing model: {model_name}")
    input_size = model_info["input_size"]
    preprocess_fn = model_info["preprocess"]
    model_fn = model_info["model_fn"]

    # Prepare datasets for the current model
    train_ds_mod = preprocess_dataset(ds_train, input_size, preprocess_fn, batch_size=BATCH_SIZE, shuffle=True)
    val_ds_mod = preprocess_dataset(ds_val, input_size, preprocess_fn, batch_size=BATCH_SIZE, shuffle=False)

    # Build, compile, and train the model
    model = build_model(model_fn, input_size, num_classes)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    print("Training...")
    history = model.fit(train_ds_mod, epochs=EPOCHS, validation_data=val_ds_mod)

    # Evaluate the model on validation set
    loss, accuracy = model.evaluate(val_ds_mod)
    print(f"{model_name} Validation Accuracy: {accuracy*100:.2f}%")

    # Test on a "tricky" image (with added noise)
    orig_pred, noisy_pred = test_tricky_image(model, input_size, preprocess_fn)
    print(f"Original prediction: {orig_pred}, Noisy prediction: {noisy_pred}")

    results_summary.append({
        'Model': model_name,
        'Test Accuracy': accuracy,
        'Original Prediction': orig_pred,
        'Noisy Prediction': noisy_pred
    })

# Display results in a table.
df = pd.DataFrame(results_summary)
df['Test Accuracy'] = df['Test Accuracy'].apply(lambda x: f"{x*100:.2f}%")
print("\nComparison of Model Performance:")
print(df.to_string(index=False))


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
Classes: ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

Processing model: VGG16


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_8           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 5)                   │           2,5

 Total params: 14,717,253 (56.14 MB)

...

 Total params: 21,813,029 (83.21 MB)

 Trainable params: 21,778,597 (83.08 MB)

 Non-trainable params: 34,432 (134.50 KB)

Training...
Epoch 1/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 112s 556ms/step - accuracy: 0.7075 - loss: 0.8142 - val_accuracy: 0.1866 - val_loss: 128.3768
Epoch 2/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 7s 76ms/step - accuracy: 0.8472 - loss: 0.4235 - val_accuracy: 0.7166 - val_loss: 0.9102
Epoch 3/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 7s 75ms/step - accuracy: 0.8760 - loss: 0.3458 - val_accuracy: 0.7221 - val_loss: 1.4822
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7281 - loss: 1.4109
InceptionV3 Validation Accuracy: 72.21%


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Original prediction: tulips, Noisy prediction: tulips

Comparison of Model Performance:
      Model Test Accuracy Original Prediction Noisy Prediction
      VGG16        27.38%              tulips           tulips
   ResNet50        78.07%              tulips           tulips
InceptionV3        72.21%              tulips           tulips


## **Task 4-2**

In [ ]:
!pip install tensorflow tensorflow-datasets opencv-python pandas
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# -------------------------
# Configuration
# -------------------------
BATCH_SIZE = 32
EPOCHS = 3   # For demonstration; increase for better performance
SEED = 123

# Load the public tf_flowers dataset (5 flower classes)
(ds_train, ds_val), ds_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)
class_names = ds_info.features['label'].names  # e.g. ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
num_classes = len(class_names)
print("Classes:", class_names)

# -------------------------
# Preprocessing Function
# -------------------------
def preprocess_dataset(ds, input_size, preprocess_fn, batch_size=BATCH_SIZE, shuffle=False):
    """
    Resizes images to input_size, casts them to float32,
    and applies the given preprocessing function.
    """
    def _preprocess(image, label):
        image = tf.image.resize(image, input_size)
        image = tf.cast(image, tf.float32)
        image = preprocess_fn(image)
        return image, label
    ds = ds.map(_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(1000, seed=SEED)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

# -------------------------
# Model Building Function
# -------------------------
def build_model_transfer(model_fn, input_size, num_classes, freeze_base=True):
    """
    Builds a transfer learning model using the specified Keras Applications model.
    If freeze_base is True, the base model’s layers are frozen.
    """
    inp = Input(shape=(*input_size, 3))
    base_model = model_fn(weights='imagenet', include_top=False, input_tensor=inp)
    base_model.trainable = not freeze_base  # Freeze if freeze_base is True
    x = GlobalAveragePooling2D()(base_model.output)
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inp, outputs=out)
    return model

# -------------------------
# Define Models Dictionary
# -------------------------
# For each model, we define its expected input size and its preprocessing function.
models_dict = {
    "VGG16": {
         "model_fn": VGG16,
         "input_size": (224, 224),
         "preprocess": vgg16_preprocess
    },
    "ResNet50": {
         "model_fn": ResNet50,
         "input_size": (224, 224),
         "preprocess": resnet50_preprocess
    },
    "InceptionV3": {
         "model_fn": InceptionV3,
         "input_size": (299, 299),
         "preprocess": inception_preprocess
    }
}

# -------------------------
# Main Pipeline: Experiments
# -------------------------
results = []

for model_name, info in models_dict.items():
    input_size = info["input_size"]
    preprocess_fn = info["preprocess"]
    model_fn = info["model_fn"]

    print(f"\nProcessing {model_name} (input size: {input_size})")

    # Prepare training and validation datasets for the current model
    train_ds_mod = preprocess_dataset(ds_train, input_size, preprocess_fn, batch_size=BATCH_SIZE, shuffle=True)
    val_ds_mod = preprocess_dataset(ds_val, input_size, preprocess_fn, batch_size=BATCH_SIZE, shuffle=False)

    # Experiment 1: Frozen Base
    print("Experiment: Frozen Base")
    model_frozen = build_model_transfer(model_fn, input_size, num_classes, freeze_base=True)
    model_frozen.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model_frozen.fit(train_ds_mod, epochs=EPOCHS, validation_data=val_ds_mod, verbose=2)
    loss_frozen, acc_frozen = model_frozen.evaluate(val_ds_mod, verbose=0)
    results.append({"Model": model_name, "Setup": "Frozen", "Validation Accuracy": acc_frozen})

    # Experiment 2: Fine-tuned Base
    print("Experiment: Fine-tuned Base")
    model_finetuned = build_model_transfer(model_fn, input_size, num_classes, freeze_base=False)
    # Use a lower learning rate for fine-tuning
    model_finetuned.compile(optimizer=Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model_finetuned.fit(train_ds_mod, epochs=EPOCHS, validation_data=val_ds_mod, verbose=2)
    loss_finetuned, acc_finetuned = model_finetuned.evaluate(val_ds_mod, verbose=0)
    results.append({"Model": model_name, "Setup": "Fine-tuned", "Validation Accuracy": acc_finetuned})

# Present the results in a table
df = pd.DataFrame(results)
df['Validation Accuracy'] = df['Validation Accuracy'].apply(lambda x: f"{x*100:.2f}%")
print("\nComparison of Model Performance:")
print(df.to_string(index=False))


Classes: ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

Processing VGG16 (input size: (224, 224))
Experiment: Frozen Base
Epoch 1/3
92/92 - 9s - 94ms/step - accuracy: 0.5480 - loss: 2.5588 - val_accuracy: 0.7193 - val_loss: 1.2309
Epoch 2/3
92/92 - 2s - 26ms/step - accuracy: 0.7670 - loss: 0.9326 - val_accuracy: 0.8120 - val_loss: 0.8377
Epoch 3/3
92/92 - 2s - 26ms/step - accuracy: 0.8171 - loss: 0.6488 - val_accuracy: 0.8283 - val_loss: 0.7091
Experiment: Fine-tuned Base
Epoch 1/3
92/92 - 16s - 169ms/step - accuracy: 0.5623 - loss: 1.3958 - val_accuracy: 0.8038 - val_loss: 0.5974
Epoch 2/3
92/92 - 6s - 63ms/step - accuracy: 0.8658 - loss: 0.3957 - val_accuracy: 0.8556 - val_loss: 0.4157
Epoch 3/3
92/92 - 6s - 63ms/step - accuracy: 0.9329 - loss: 0.1976 - val_accuracy: 0.8760 - val_loss: 0.3540

Processing ResNet50 (input size: (224, 224))
Experiment: Frozen Base
Epoch 1/3
92/92 - 20s - 213ms/step - accuracy: 0.7595 - loss: 0.6617 - val_accuracy: 0.8801 - val_loss: 0.3416
Epo

## **Task 3-3**

In [ ]:
# Install required packages in Colab (if not installed):
!pip install tensorflow tensorflow-datasets flask pyngrok opencv-python-headless pandas

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import cv2
import os
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam

# -------------------------
# PART 1: MODEL TRAINING
# -------------------------
print("Loading tf_flowers dataset...")
(ds_train, ds_val), ds_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)
class_names = ds_info.features['label'].names  # e.g., ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
num_classes = len(class_names)
print("Classes:", class_names)

BATCH_SIZE = 32
EPOCHS = 3   # For demonstration; increase for better performance
SEED = 123

def preprocess_dataset(ds, input_size, preprocess_fn, batch_size=BATCH_SIZE, shuffle=False):
    """Resize, cast, and preprocess images."""
    def _preprocess(image, label):
        image = tf.image.resize(image, input_size)
        image = tf.cast(image, tf.float32)
        image = preprocess_fn(image)
        return image, label
    ds = ds.map(_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(1000, seed=SEED)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

# Using VGG16 as our base; input size is (224, 224)
input_size = (224, 224)
train_ds = preprocess_dataset(ds_train, input_size, vgg16_preprocess, batch_size=BATCH_SIZE, shuffle=True)
val_ds = preprocess_dataset(ds_val, input_size, vgg16_preprocess, batch_size=BATCH_SIZE, shuffle=False)

def build_transfer_model(input_size, num_classes, freeze_base=True):
    """Builds a transfer learning model using VGG16."""
    inp = Input(shape=(*input_size, 3))
    base_model = VGG16(weights='imagenet', include_top=False, input_tensor=inp)
    base_model.trainable = not freeze_base  # Freeze if freeze_base is True
    x = GlobalAveragePooling2D()(base_model.output)
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inp, outputs=out)
    return model

# Experiment: Fine-tuned Base (for best performance in this example)
print("\nTraining model with fine-tuned base layers...")
model_finetuned = build_transfer_model(input_size, num_classes, freeze_base=False)
# Use a lower learning rate for fine-tuning
model_finetuned.compile(optimizer=Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_finetuned.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, verbose=2)
loss_ft, acc_ft = model_finetuned.evaluate(val_ds, verbose=0)
print(f"Fine-tuned base validation accuracy: {acc_ft*100:.2f}%")

# Save the fine-tuned model using the native Keras format (.keras)
MODEL_PATH = "flower_model.keras"
model_finetuned.save(MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")

# -------------------------
# PART 2: BUILDING THE FLASK RESTFUL SERVICE
# -------------------------
from flask import Flask, request, jsonify
from pyngrok import ngrok
from PIL import Image
import io

app = Flask(__name__)

# Load the saved model
model = load_model(MODEL_PATH)
preprocess_fn = vgg16_preprocess  # same as during training
TARGET_SIZE = input_size  # (224, 224)

@app.route('/')
def index():
    return '''
    <!doctype html>
    <title>Flower Classification Service</title>
    <h1>Upload an image for flower classification</h1>
    <form method="POST" action="/predict" enctype="multipart/form-data">
      <input type="file" name="file" accept="image/*">
      <input type="submit" value="Predict">
    </form>
    '''

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({"error": "No file part in the request"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400
    try:
        img = Image.open(file.stream).convert("RGB")
    except Exception as e:
        return jsonify({"error": f"Could not process the image: {str(e)}"}), 400

    img = img.resize(TARGET_SIZE)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0).astype(np.float32)
    img_preprocessed = preprocess_fn(img_array)
    preds = model.predict(img_preprocessed)
    pred_class = class_names[np.argmax(preds)]
    return jsonify({"predicted_class": pred_class})

# -------------------------
# PART 3: RUNNING THE FLASK APP WITH NGROK (for Google Colab)
# -------------------------
# IMPORTANT: Replace "YOUR_NGROK_AUTHTOKEN_HERE" with your actual ngrok authtoken.
from pyngrok import conf
conf.get_default().auth_token = "YOUR_NGROK_AUTHTOKEN_HERE"  # <-- Set your ngrok authtoken here

# Start an ngrok tunnel to the Flask app (port 5000)
public_url = ngrok.connect(5000).public_url
print(f"ngrok tunnel URL: {public_url}")

# Run the Flask app
app.run()


Loading tf_flowers dataset...
Classes: ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

Training model with fine-tuned base layers...
Epoch 1/3
92/92 - 16s - 172ms/step - accuracy: 0.7115 - loss: 0.8814 - val_accuracy: 0.8338 - val_loss: 0.4456
Epoch 2/3
92/92 - 6s - 63ms/step - accuracy: 0.9033 - loss: 0.2715 - val_accuracy: 0.8965 - val_loss: 0.3215
Epoch 3/3
92/92 - 6s - 63ms/step - accuracy: 0.9659 - loss: 0.1144 - val_accuracy: 0.8965 - val_loss: 0.3520
Fine-tuned base validation accuracy: 89.65%
Model saved to flower_model.keras


ERROR:pyngrok.process.ngrok:t=2025-02-26T16:19:51+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_NGROK_AUTHTOKEN_HERE\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-02-26T16:19:51+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_NGROK_AUTHTOKEN_HERE\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-02-26T16:19:51+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: The authtoken you specified does not look l

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_NGROK_AUTHTOKEN_HERE\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

## **Task 3-4 (Extra)**

## **Task 3-5 (Extra)**

In [ ]:
!pip install tensorflow tensorflow-datasets opencv-python-headless pandas


In [ ]:
import ssl
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import os
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.layers import TimeDistributed, GlobalAveragePooling2D, GlobalAveragePooling1D, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Disable SSL certificate verification (for TFDS downloads in Colab)
ssl._create_default_https_context = ssl._create_unverified_context

# -------------------------
# Configuration
# -------------------------
NUM_FRAMES = 8            # Number of frames to sample per video
BATCH_SIZE = 2            # Small batch size for demonstration
EPOCHS = 3                # Few epochs for demonstration; adjust as needed
SEED = 123

# -------------------------
# Load UCF101 Dataset (restricted subset for speed)
# -------------------------
print("Loading UCF101 dataset...")
(ds_train, ds_val), ds_info = tfds.load(
    'ucf101',
    split=['train[:10%]', 'test[:10%]'],  # Use only 10% of each split for speed
    as_supervised=True,
    with_info=True
)
# Further restrict to a very small number of examples for demonstration:
ds_train = ds_train.take(20)
ds_val = ds_val.take(10)

# UCF101 has 101 classes
class_names = ds_info.features['label'].names
num_classes = len(class_names)
print("Number of classes:", num_classes)
# Uncomment the following line to see full class names:
# print("Classes:", class_names)

# -------------------------
# Video Preprocessing Functions
# -------------------------
def sample_frames(video, num_frames):
    """Uniformly sample 'num_frames' frames from the video tensor."""
    T = tf.shape(video)[0]
    indices = tf.linspace(0.0, tf.cast(T - 1, tf.float32), num_frames)
    indices = tf.cast(indices, tf.int32)
    sampled = tf.gather(video, indices)
    return sampled

def preprocess_video(video, label, input_size, num_frames, preprocess_fn):
    """
    Samples a fixed number of frames from the video, resizes each frame to input_size,
    casts frames to float32, and applies the specified image preprocessing function.
    """
    video = sample_frames(video, num_frames)  # shape: (num_frames, H, W, 3)
    video = tf.image.resize(video, input_size)
    video = tf.cast(video, tf.float32)
    video = preprocess_fn(video)  # apply preprocessing to each frame
    return video, label

def prepare_video_dataset(ds, input_size, preprocess_fn, num_frames=NUM_FRAMES, batch_size=BATCH_SIZE, shuffle=False):
    """
    Preprocesses the video dataset by sampling frames, resizing, and applying the image preprocessing.
    Then batches and prefetches the dataset.
    """
    ds = ds.map(lambda video, label: preprocess_video(video, label, input_size, num_frames, preprocess_fn),
                num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(1000, seed=SEED)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

# -------------------------
# Build Video Classification Model
# -------------------------
def build_video_model(model_fn, input_size, num_frames, num_classes, freeze_base=True):
    """
    Builds a video classification model:
      - Input shape: (num_frames, H, W, 3)
      - Each frame is processed by the pre-trained image model (wrapped in TimeDistributed)
      - Frame-level features are aggregated over time using GlobalAveragePooling1D
      - Final Dense softmax layer outputs the class probabilities
    """
    inp = Input(shape=(num_frames, *input_size, 3))
    base_model = model_fn(weights='imagenet', include_top=False)
    base_model.trainable = not freeze_base
    x = TimeDistributed(base_model)(inp)  # Process each frame
    x = TimeDistributed(GlobalAveragePooling2D())(x)  # (batch, num_frames, features)
    x = GlobalAveragePooling1D()(x)  # Aggregate over time (batch, features)
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inp, outputs=out)
    return model

# -------------------------
# Define Models Dictionary
# -------------------------
# For each model, define its expected input size and corresponding image preprocessing function.
models_dict = {
    "VGG16": {
         "model_fn": VGG16,
         "input_size": (224, 224),
         "preprocess": vgg16_preprocess
    },
    "ResNet50": {
         "model_fn": ResNet50,
         "input_size": (224, 224),
         "preprocess": resnet50_preprocess
    },
    "InceptionV3": {
         "model_fn": InceptionV3,
         "input_size": (299, 299),
         "preprocess": inception_preprocess
    }
}

# -------------------------
# Main Pipeline: Train and Evaluate Models
# -------------------------
results = []

for model_name, info in models_dict.items():
    input_size = info["input_size"]
    preprocess_fn = info["preprocess"]
    model_fn = info["model_fn"]

    print(f"\nProcessing model: {model_name} (input size: {input_size})")

    # Prepare training and validation datasets for the current model
    train_ds_mod = prepare_video_dataset(ds_train, input_size, preprocess_fn, num_frames=NUM_FRAMES, batch_size=BATCH_SIZE, shuffle=True)
    val_ds_mod = prepare_video_dataset(ds_val, input_size, preprocess_fn, num_frames=NUM_FRAMES, batch_size=BATCH_SIZE, shuffle=False)

    # Build the video model (using fine-tuning: unfreeze base layers)
    model = build_video_model(model_fn, input_size, NUM_FRAMES, num_classes, freeze_base=False)
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    print("Training...")
    model.fit(train_ds_mod, epochs=EPOCHS, validation_data=val_ds_mod, verbose=2)

    # Evaluate on validation set
    loss, acc = model.evaluate(val_ds_mod, verbose=0)
    print(f"{model_name} validation accuracy: {acc*100:.2f}%")
    results.append({"Model": model_name, "Validation Accuracy": acc})

# Present the results in a table
df = pd.DataFrame(results)
df["Validation Accuracy"] = df["Validation Accuracy"].apply(lambda x: f"{x*100:.2f}%")
print("\nComparison of Model Performance:")
print(df.to_string(index=False))


Loading UCF101 dataset...


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

SSLError: HTTPSConnectionPool(host='www.crcv.ucf.edu', port=443): Max retries exceeded with url: /data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics import classification_report

# ----------------------------
# Parameters & Configuration
# ----------------------------
BATCH_SIZE = 4      # smaller batch size for demo purposes
EPOCHS = 5          # fewer epochs for quick demonstration
IMG_SIZE = (224, 224)
NUM_FRAMES = 16     # fixed number of frames per video

# ----------------------------
# Load Public Dataset: UCF101
# ----------------------------
# For demonstration, we use a very small fraction of the dataset.
# The UCF101 dataset contains 101 classes.
print("Loading UCF101 dataset (a small fraction for demo)...")
ds_train = tfds.load("ucf101", split="train[:1%]", shuffle_files=True, as_supervised=False)
ds_val   = tfds.load("ucf101", split="train[1%:2%]", shuffle_files=True, as_supervised=False)

# Retrieve dataset info for number of classes and class names.
dataset_info = tfds.builder("ucf101").info
num_classes = dataset_info.features["label"].num_classes
class_names = dataset_info.features["label"].names
print(f"Number of classes: {num_classes}")

# ----------------------------
# Video Preprocessing Function
# ----------------------------
def process_video(example):
    """
    Preprocess a single example:
    - Ensures a fixed number of frames by padding if necessary.
    - Evenly samples NUM_FRAMES frames.
    - Resizes frames to IMG_SIZE.
    - Normalizes pixel values.
    """
    video = example["video"]   # video shape: (num_frames, H, W, C)
    label = example["label"]

    num_frames_in_video = tf.shape(video)[0]

    # If video has fewer frames than NUM_FRAMES, pad with the last frame.
    def pad_video():
        pad_len = NUM_FRAMES - num_frames_in_video
        last_frame = video[-1:]
        padding = tf.repeat(last_frame, pad_len, axis=0)
        return tf.concat([video, padding], axis=0)

    video = tf.cond(num_frames_in_video < NUM_FRAMES, pad_video, lambda: video)

    # Evenly sample NUM_FRAMES frames.
    indices = tf.linspace(0.0, tf.cast(tf.shape(video)[0] - 1, tf.float32), NUM_FRAMES)
    indices = tf.cast(indices, tf.int32)
    video = tf.gather(video, indices)

    # Resize and normalize frames.
    video = tf.image.resize(video, IMG_SIZE)
    video = tf.cast(video, tf.float32) / 255.0
    return video, label

# Apply the preprocessing function in parallel.
ds_train = ds_train.map(process_video, num_parallel_calls=tf.data.AUTOTUNE)
ds_val   = ds_val.map(process_video, num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle, batch, and prefetch the datasets.
ds_train = ds_train.shuffle(100).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_val   = ds_val.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# ----------------------------
# Build the Baseline Model
# ----------------------------
def build_baseline_model(input_shape=(NUM_FRAMES, IMG_SIZE[0], IMG_SIZE[1], 3), num_classes=num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv3D(32, kernel_size=(3, 3, 3), activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = tf.keras.layers.Conv3D(64, kernel_size=(3, 3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = tf.keras.layers.GlobalAveragePooling3D()(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, outputs)

baseline_model = build_baseline_model()
baseline_model.compile(optimizer='adam',
                         loss='sparse_categorical_crossentropy',
                         metrics=['accuracy'])
baseline_model.summary()

print("\nTraining Baseline Model:")
baseline_history = baseline_model.fit(ds_train,
                                      epochs=EPOCHS,
                                      validation_data=ds_val)

# ----------------------------
# Build the Transfer Learning Model with Movinet
# ----------------------------
# Use a pre-trained Movinet model from TensorFlow Hub (trained on Kinetics-600).
movinet_url = "https://tfhub.dev/tensorflow/movinet/a0/kinetics-600/classification/3"
movinet_layer = hub.KerasLayer(movinet_url, trainable=False, name="movinet_layer")

def build_movinet_model(input_shape=(NUM_FRAMES, IMG_SIZE[0], IMG_SIZE[1], 3), num_classes=num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    # Pass the video through the Movinet layer.
    x = movinet_layer(inputs)
    # Add a new dense layer to adapt to the target number of classes.
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, outputs)

movinet_model = build_movinet_model()
movinet_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])
movinet_model.summary()

print("\nTraining Movinet Transfer Learning Model:")
movinet_history = movinet_model.fit(ds_train,
                                    epochs=EPOCHS,
                                    validation_data=ds_val)

# ----------------------------
# Evaluation & Results
# ----------------------------
print("\nEvaluating Models on Validation Set:")
baseline_eval = baseline_model.evaluate(ds_val, verbose=0)
movinet_eval = movinet_model.evaluate(ds_val, verbose=0)

print(f"Baseline model accuracy: {baseline_eval[1]*100:.2f}%")
print(f"Movinet transfer learning model accuracy: {movinet_eval[1]*100:.2f}%")

# Optional: Generate per-class classification reports.
def get_predictions(model, dataset):
    preds = []
    true_labels = []
    for videos, labels in dataset:
        batch_preds = model.predict(videos)
        preds.extend(np.argmax(batch_preds, axis=1))
        true_labels.extend(labels.numpy())
    return preds, true_labels

baseline_preds, baseline_true = get_predictions(baseline_model, ds_val)
movinet_preds, movinet_true = get_predictions(movinet_model, ds_val)

print("\nBaseline Model Classification Report:")
print(classification_report(baseline_true, baseline_preds, target_names=class_names))
print("Movinet Model Classification Report:")
print(classification_report(movinet_true, movinet_preds, target_names=class_names))


Loading UCF101 dataset (a small fraction for demo)...


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

SSLError: HTTPSConnectionPool(host='www.crcv.ucf.edu', port=443): Max retries exceeded with url: /data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

In [ ]:
!pip install tensorflow tensorflow_hub tensorflow_datasets tensorflow-io

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_io as tfio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)

# Parameters
NUM_FRAMES = 16
IMG_SIZE = 128
BATCH_SIZE = 2
EPOCHS = 5

###############################################################################
# 1) Load & filter UCF101 dataset from TFDS
###############################################################################
def filter_two_classes(sample):
    # Keep only label == 0 (ApplyEyeMakeup) or label == 1 (Archery)
    label = sample['label']
    return tf.logical_or(tf.equal(label, 0), tf.equal(label, 1))

ds_train_full = tfds.load("ucf101", split="train", shuffle_files=True)
ds_train_filtered = ds_train_full.filter(filter_two_classes)
# Limit to smaller subset for speed
ds_train_filtered = ds_train_filtered.shuffle(1000).take(80)

ds_val_full = tfds.load("ucf101", split="test", shuffle_files=False)
ds_val_filtered = ds_val_full.filter(filter_two_classes)
ds_val_filtered = ds_val_filtered.shuffle(500).take(20)

###############################################################################
# 2) Preprocessing (sample frames & resize)
###############################################################################
def sample_frames(video, num_frames=NUM_FRAMES):
    video = tf.cast(video, tf.float32)
    total_frames = tf.shape(video)[0]

    def pad_video():
        pad_len = num_frames - total_frames
        padding = tf.zeros((pad_len, tf.shape(video)[1], tf.shape(video)[2], tf.shape(video)[3]))
        return tf.concat([video, padding], axis=0)

    video = tf.cond(total_frames < num_frames, pad_video, lambda: video)
    total_frames = tf.shape(video)[0]

    indices = tf.linspace(0.0, tf.cast(total_frames - 1, tf.float32), num_frames)
    indices = tf.cast(indices, tf.int32)
    video = tf.gather(video, indices)
    return video

def preprocess(sample):
    video = sample['video']
    label = sample['label']
    video = sample_frames(video, NUM_FRAMES)
    video = tf.image.resize(video, (IMG_SIZE, IMG_SIZE))
    return video, label

###############################################################################
# 3) Build tf.data pipelines
###############################################################################
AUTO = tf.data.AUTOTUNE

train_ds = (
    ds_train_filtered
    .map(preprocess, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

val_ds = (
    ds_val_filtered
    .map(preprocess, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

###############################################################################
# 4) Build the MoViNet transfer learning model
###############################################################################
num_classes = 2
hub_url = "https://tfhub.dev/tensorflow/movinet/a0/base/kinetics-600/classification/3"
base_model = hub.KerasLayer(hub_url, trainable=False)

inputs = tf.keras.Input([NUM_FRAMES, IMG_SIZE, IMG_SIZE, 3])
x = base_model(inputs, training=False)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"]
)

model.summary()

###############################################################################
# 5) Train
###############################################################################
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

###############################################################################
# 6) Evaluate & show results in a table
###############################################################################
val_loss, val_acc = model.evaluate(val_ds)
print("Validation loss:", val_loss)
print("Validation accuracy:", val_acc)

hist_df = pd.DataFrame(history.history)
hist_df['epoch'] = hist_df.index + 1
hist_df.rename(columns={
    'accuracy': 'Train Accuracy',
    'val_accuracy': 'Val Accuracy',
    'loss': 'Train Loss',
    'val_loss': 'Val Loss'
}, inplace=True)
print(hist_df[['epoch', 'Train Accuracy', 'Val Accuracy', 'Train Loss', 'Val Loss']])

###############################################################################
# 7) Optional: Inspect predictions
###############################################################################
label_names = ["ApplyEyeMakeup", "Archery"]

for videos, labs in val_ds.take(1):
    preds = model.predict(videos)
    for i, pred in enumerate(preds):
        true_label = label_names[labs[i].numpy()]
        predicted_label = label_names[np.argmax(pred)]
        print(f"Video {i}: True = {true_label}, Pred = {predicted_label}, Raw Softmax = {pred}")


TensorFlow version: 2.18.0


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

SSLError: HTTPSConnectionPool(host='www.crcv.ucf.edu', port=443): Max retries exceeded with url: /data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))